In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from sklearn.metrics import mean_squared_error
from numba import jit, cuda
warnings.filterwarnings("ignore")
import os
import joblib
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
models=[]

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

df = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')
x_test = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv')

In [ ]:
targets = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv')

In [ ]:
from sklearn.preprocessing import StandardScaler

def generic_preprocess(x):
    x['imbalance_ratio'] = x['imbalance_size'] / x['matched_size']
    x['bid_ask_volume_diff'] = x['ask_size'] - x['bid_size']
    x['mid_price'] = (x['ask_price'] + x['bid_price']) / 2
    x['bid_plus_ask_sizes'] = x['bid_size'] + x['ask_size']
    if 'time_id' not in x.columns:
        x['time_id']=[0]*x.shape[0]
    x['far_price'].fillna(1)
    x['near_price'].fillna(1)
    x.drop(['imbalance_size','matched_size','ask_size','bid_size','row_id','time_id'],axis=1,inplace=True)
    for i in x.columns:
        if x[i].isnull().sum()>0:
            x[i].fillna(x[i].mean(),inplace=True)
    return x



def standardize(df):
    #Standardize by stock ID (each stock is fit to have SD=1 and mean 0)
    def standardize_stock(group):
        columns_to_scale = group.columns.difference(['stock_id', 'date_id', 'seconds_in_bucket'])
        scaler = StandardScaler()
        group[columns_to_scale] = scaler.fit_transform(group[columns_to_scale])
        return group

    # Reset index if 'stock_id' is part of the index
    if 'stock_id' in df.index.names:
        df = df.reset_index()

    # Apply standardization to each stock group
    df_scaled = df.groupby('stock_id').apply(standardize_stock)

    # Reset index to flatten the DataFrame after groupby
    df_scaled = df_scaled.reset_index(drop=True)

    # Sort by 'stock_id', 'date_id', 'seconds_in_bucket'


    return df_scaled

def rolling_average(df):
    # Define a function to apply the rolling average within each group
    def apply_rolling(group):
        # Calculate rolling average without looking ahead
        rolling_avg = group['reference_price'].rolling(window=10, min_periods=1).mean()
        # Fill NaN values (for the first 9 elements in each group) with 1
        rolling_avg.iloc[:9] = 0
        return rolling_avg

    # Group by 'stock_id' and 'date_id', then apply the rolling function
    df['rolling_avg'] = df.groupby(['stock_id', 'date_id']).apply(apply_rolling).reset_index(level=[0,1], drop=True)

    return df
    

In [ ]:
def data_preprocessing(df):
    df1 = generic_preprocessing(df)
    df2 = standardize(df1)
    df3 = rolling_average(df3)
    return df3

In [ ]:
x = df.drop(['target'],axis=1)
x = data_preprocessing(x)
x.head()

In [ ]:
df_test = x_test.copy()
x=df.drop(['target'],axis=1)
x = x[x['date_id'] < 478]

y=df[['date_id','target']]
y = y[y['date_id'] < 478].fillna(0,inplace=True)